In [142]:
# Create a dataset that contains the monthly means of Sea Surface Temperature anomalies and total column water vapor from Jan 1979-Dec 2023 over the Pacific Basin (65°N to 65°S, 120°E to 60°W)
 #masked out over land - save this to your computer.
 #Plot maps of the mean SST and mean total column water vapor for the entire period of record.

In [143]:
from dask.distributed import Client, progress
client = Client(dashboard_address=':5555')  
client

C:\Users\spantha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\distributed\node.py:182: UserWarning: Port 5555 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62564 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62564/status,
Dashboard: http://127.0.0.1:62564/status,Workers: 4
Total threads: 8,Total memory: 7.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62567,Workers: 4
Dashboard: http://127.0.0.1:62564/status,Total threads: 8
Started: Just now,Total memory: 7.86 GiB
Comm: tcp://127.0.0.1:62591,Total threads: 2
Dashboard: http://127.0.0.1:62593/status,Memory: 1.97 GiB
Nanny: tcp://127.0.0.1:62570,


In [144]:
# Import libraries

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os

In [145]:
filename = 'era5data_pacificbasin_sst_vapor.nc'

if os.path.exists(filename):
    # Read saved data
    ds= xr.open_dataset(filename)

else:
    # Pull the data only if not available locally
    ds = xr.open_dataset('https://thredds.rda.ucar.edu/thredds/dodsC/aggregations/g/ds633.1/2/TP',
                     chunks={'time':'500MB'})

    variables = ['Total_column_water_vapour_surface_Mixed_intervals_Average',
        'Sea_surface_temperature_surface_Mixed_intervals_Average']
    #select the two needed variables every 4th point to get 1 degree resolution
    ds = ds[variables].sel(lat=slice(65, -65, 4), lon=slice(120, 300, 4))
    ds.to_netcdf(filename)

In [146]:
ds

<xarray.Dataset>
Dimensions:                                                    (time: 516,
                                                                lat: 130,
                                                                lon: 181)
Coordinates:
  * lat                                                        (lat) float32 ...
  * lon                                                        (lon) float32 ...
  * time                                                       (time) datetime64[ns] ...
    reftime                                                    (time) datetime64[ns] ...
Data variables:
    Total_column_water_vapour_surface_Mixed_intervals_Average  (time, lat, lon) float32 ...
    Sea_surface_temperature_surface_Mixed_intervals_Average    (time, lat, lon) float32 ...
Attributes:
    Originating_or_generating_Center:     European Centre for Medium Range We...
    Originating_or_generating_Subcenter:  0
    GRIB_table_version:                   0,128
    file_format:                          GRIB-1
    Conventions:                          CF-1.6
    history:                              Read using CDM IOSP GribCollection v3
    featureType:                          GRID
    _CoordSysBuilder:                     ucar.nc2.dataset.conv.CF1Convention

Jan 1979-Dec 2023

In [147]:
ds_mean = ds.mean(dim=['time'])

In [148]:
ds_mean

<xarray.Dataset>
Dimensions:                                                    (lat: 130,
                                                                lon: 181)
Coordinates:
  * lat                                                        (lat) float32 ...
  * lon                                                        (lon) float32 ...
Data variables:
    Total_column_water_vapour_surface_Mixed_intervals_Average  (lat, lon) float32 ...
    Sea_surface_temperature_surface_Mixed_intervals_Average    (lat, lon) float32 ...

In [149]:


url = "https://thredds.rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.invariant/197901/e5.oper.invariant.128_172_lsm.ll025sc.1979010100_1979010100.nc"

ds_lsm = xr.open_dataset(url)
print(ds_lsm)



<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 1)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1979-01-01
Data variables:
    utc_date   (time) int32 ...
    LSM        (time, latitude, longitude) float32 ...
Attributes:
    _NCProperties:                   version=1|netcdflibversion=4.6.1|hdf5lib...
    DATA_SOURCE:                     ECMWF: https://cds.climate.copernicus.eu...
    NETCDF_CONVERSION:               CISL RDA: Conversion from ECMWF GRIB1 da...
    NETCDF_VERSION:                  4.6.1
    CONVERSION_PLATFORM:             Linux casper04 3.10.0-693.21.1.el7.x86_6...
    CONVERSION_DATE:                 Mon May 13 18:10:35 MDT 2019
    Conventions:                     CF-1.6
    NETCDF_COMPRESSION:              NCO: Precision-preserving compression to...
    history:                    

In [150]:
ds_lsm

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 1)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1979-01-01
Data variables:
    utc_date   (time) int32 ...
    LSM        (time, latitude, longitude) float32 ...
Attributes:
    _NCProperties:                   version=1|netcdflibversion=4.6.1|hdf5lib...
    DATA_SOURCE:                     ECMWF: https://cds.climate.copernicus.eu...
    NETCDF_CONVERSION:               CISL RDA: Conversion from ECMWF GRIB1 da...
    NETCDF_VERSION:                  4.6.1
    CONVERSION_PLATFORM:             Linux casper04 3.10.0-693.21.1.el7.x86_6...
    CONVERSION_DATE:                 Mon May 13 18:10:35 MDT 2019
    Conventions:                     CF-1.6
    NETCDF_COMPRESSION:              NCO: Precision-preserving compression to...
    history:                         Mon May 13 18:10:35 2019: ncks -4 --ppc ...
    NCO:                             netCDF Operators version 4.7.4 (http://n...
    DODS_EXTRA.Unlimited_Dimension:  time

In [151]:
ds_lsm = ds_lsm.sel(latitude=slice(65, -65, 4), longitude=slice(120, 300, 4))

ds_lsm = ds_lsm.mean(dim=['time'])


In [155]:
ds_lsm

<xarray.Dataset>
Dimensions:    (latitude: 131, longitude: 181)
Coordinates:
  * latitude   (latitude) float64 65.0 64.0 63.0 62.0 ... -63.0 -64.0 -65.0
  * longitude  (longitude) float64 120.0 121.0 122.0 123.0 ... 298.0 299.0 300.0
Data variables:
    utc_date   float64 1.979e+09
    LSM        (latitude, longitude) float32 0.9839 0.9778 ... 0.9999 0.9546

In [153]:
ds_mean

<xarray.Dataset>
Dimensions:                                                    (lat: 130,
                                                                lon: 181)
Coordinates:
  * lat                                                        (lat) float32 ...
  * lon                                                        (lon) float32 ...
Data variables:
    Total_column_water_vapour_surface_Mixed_intervals_Average  (lat, lon) float32 ...
    Sea_surface_temperature_surface_Mixed_intervals_Average    (lat, lon) float32 ...

In [ ]:
a =ds_mean.iloc[::4,'Total_column_water_vapour_surface_Mixed_intervals_Average', ]

In [ ]:
ds_mean_lsm = ds_mean.where(ds_lsm.LSM ==0)

